In [1]:
import sys
import platform
from os.path import join, exists, abspath, dirname
from os import getcwd, makedirs
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps, cm, colors
import scipy
from scipy.stats import ttest_1samp, pearsonr
from scipy.io import loadmat
from sklearn.metrics import r2_score
import seaborn as sns
import h5py

from tqdm import tqdm

In [2]:
import nibabel as nb
from nilearn import plotting, image
from nipype.interfaces import fsl

In [3]:
dir_current = getcwd().replace('\\','/')

tmp = dir_current.split('/')
idx = [ii for ii, s in enumerate(tmp) if s=='github'][0]

dir_git = '/'.join(tmp[:idx+1])
dir_git

'/home/sungbeenpark/github'

In [4]:
dname = join(dir_git,'nitools')
sys.path.append(dname)
import nitools as nt

In [5]:
dname = join(dir_git,'SUITPy')
sys.path.append(dname)
import SUITPy as suit

In [6]:
dname = join(dir_git)
sys.path.append(dname)
import surfAnalysisPy as surf

In [7]:
dname = join(dir_git,'Functional_Fusion')
sys.path.append(dname)
import Functional_Fusion.atlas_map as am
import Functional_Fusion.reliability as rel

In [8]:
dname = join(dir_git,'fracridge')
sys.path.append(dname)
from fracridge import fracridge

In [9]:
dname = join(dir_git,'GLMsingle')
sys.path.append(dname)
import glmsingle
from glmsingle.glmsingle import getcanonicalhrflibrary
from glmsingle.utils.squish import squish

In [10]:
dname = join(dir_git,'GLMsingle')
sys.path.append(dname)
import glmsingle
from glmsingle.glmsingle import getcanonicalhrflibrary
from glmsingle.utils.squish import squish

In [11]:
dname = join(dir_git,'SeqSpatialSupp_fMRI')
sys.path.append(dname)
from SSS import deal_spm
from SSS import util as su
from SSS import plot as splt
from SSS import image as simage
from SSS import glmsingle as ssingle

---

In [12]:
glm = 1
dir_glm = ssingle.get_dir_glmsingle(glm)
dir_glm

'/mnt/f/SeqSpatialSupp_fMRI/GLMsingle/glm_1'

---

In [13]:
subj = 'S01'

In [14]:
fname = join(dir_glm, subj,'DESIGNINFO.mat')
DESIGNINFO = h5py.File(fname)
DESIGNINFO.keys()

<KeysViewHDF5 ['#refs#', 'condcounts', 'condinruns', 'design', 'designSINGLE', 'endbuffers', 'numtrialrun', 'opt', 'stimdur', 'stimorder', 'tr']>

In [15]:
designSINGLE = DESIGNINFO['designSINGLE']
designSINGLE.shape

(8, 1)

In [16]:
stimdur = DESIGNINFO['stimdur'][:].flatten()[0]
tr = DESIGNINFO['tr'][:].flatten()[0]
print(stimdur, tr)

2.0 1.0


---

In [17]:
fname = join(dir_glm, subj, 'TYPED_FITHRF_GLMDENOISE_RR.mat')
GLMsingle = h5py.File(fname)
GLMsingle.keys()

<KeysViewHDF5 ['#refs#', 'FRACvalue', 'HRFindex', 'HRFindexrun', 'R2', 'R2run', 'glmbadness', 'meanvol', 'modelmd', 'noisepool', 'pcnum', 'pcregressors', 'pcvoxels', 'rrbadness', 'scaleoffset', 'xvaltrend']>

In [18]:
#Find a voxel with highest variance explained of the ON-OFF model
R2 = GLMsingle['R2'][:]
R2.shape

(57, 116, 116)

In [19]:
idx = np.unravel_index(np.nanargmax(R2[:].flatten()),np.shape(R2))
idx

(37, 68, 82)

In [20]:
# Find voxel's betas
betas = GLMsingle['modelmd'][:]
betas.shape

(544, 57, 116, 116)

In [21]:
#find hrf estimated by GLMsingle
hrflib = getcanonicalhrflibrary(stimdur,tr).transpose()
print(hrflib.shape)
hrfii = squish(GLMsingle['HRFindex'],3)[np.nanargmax(R2.flatten())]
print(hrfii)
meansignal = squish(GLMsingle['meanvol'],3)[np.nanargmax(R2.flatten())]
print(meansignal)

(52, 20)
20.0
5688.0537


---

In [39]:
ref = designSINGLE[1,0]
ref

<HDF5 object reference>

In [45]:
Xs = DESIGNINFO[ref][:]
Xs.shape

(544, 407)

---

In [ ]:
list_sn = su.get_list_sn()

In [ ]:
glm = 1
for nn in tqdm(list_sn):
    subj = 'S'+nn
    ssingle.calc_y_hat(subj=subj, glm=glm)

---

In [ ]:
list_run = su.get_list_run()

---

In [ ]:
Xs = ssingle.get_designSINGLE(subj,glm,run=1)
Xs.shape

In [ ]:
subj = 'S01'
run = 'r01'

dir_work = join(dir_glm, subj)
fname = join(dir_work,'%s.Yhat.%s.nii'%(subj,run))
fname = join(dir_work,'tmap_(1,S).nii')
img = nb.load(fname)

In [ ]:
plotting.view_img(
    # stat_map_img=image.index_img(img, index=10)
    stat_map_img=img
)

---

In [ ]:
run = 1

In [ ]:
y_raw, affine, header = ssingle.get_y_raw(subj=subj, glm=glm, run=run)

## check the validation
assert y_raw.shape[-1] == T, f'T mismatch: X has {T} rows, y has {y_raw.shape[-1]} timepoints'

## Flattened in 2D for vectorized operations
Y_2d = y_raw.reshape(*spatial_shape, T).reshape(V, T).T  # (T, V)
del y_raw

In [ ]:
## get y_res
R = Y_2d - Y_hat  # (T, V)

In [ ]:
voxel = 400000
plt.plot(np.arange(T),Y_2d[:,voxel])
plt.plot(np.arange(T),Y_hat[:,voxel])
# plt.plot(np.arange(T),R[:,voxel])